In [1]:
%matplotlib notebook 

import numpy as np
import matplotlib.pyplot as plt
from specutils import Spectrum1D
import astropy.units as u

# this is a pretty fast Dynamic Time Warping implementation, 
# which is pip installable & used in PyKOSMOS
# https://dynamictimewarping.github.io/py-api/html/
import dtw

# current PyKOSMOS working code
import sys  
sys.path.append('/Users/james/Dropbox/research_projects/kosmos/')
from kosmos.apextract import _gaus
import kosmos

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size':12})
matplotlib.rcParams.update({'font.family':'serif'})

# Cartoon

In [ ]:
# WTF is DTW

# Tough Model

In [ ]:
# show the tough model & it works

In [ ]:
# the 2-step model, smooth, peak find etc

# KOSMOS

In [ ]:
#  solve a wavelength solution from some real data

In [ ]:
# do the stack of Neon solutions at all wavelength centers